In [4]:
# pip install clickhouse-connect

In [2]:
import clickhouse_connect
client = clickhouse_connect.get_client(host='localhost', port=8123, username='', password='')

In [7]:
client.command("create database SgxTrading")

In [8]:
client.command('''create table SgxTrading.Dim_Month
(
    Month_Key INT,
    MonthCode CHAR,
    Month INT,
    MonthName TEXT
)
    engine = MergeTree ORDER BY Month_Key
        SETTINGS index_granularity = 8192;
''')

In [10]:
client.command('''create table SgxTrading.Dim_Year
(
    Year_Key INT,
    Year INT,
)
    engine = MergeTree ORDER BY Year_Key
        SETTINGS index_granularity = 8192;''')

In [11]:
client.command('''create table SgxTrading.DerivatiesTrading
(
    idTransaction INT,
    Price FLOAT,
    Volume INT
)
    engine = MergeTree ORDER BY idTransaction
        SETTINGS index_granularity = 8192;''')

## Try insert

In [13]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]") \
            .appName("loadData") \
            .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
            .getOrCreate()

23/08/10 00:44:50 WARN Utils: Your hostname, chinhnv-Vostro-3590 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp3s0)
23/08/10 00:44:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/chinhnv/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/chinhnv/.ivy2/cache
The jars for the packages stored in: /home/chinhnv/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f71c18de-dd12-45c5-b96c-f9d817f91b14;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 454ms :: artifacts dl 14ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifac

In [14]:
file_name = 'WEBPXTICK_DT-20230808.csv'
df = spark.read.csv(f"hdfs://172.29.0.2:8020/sgx_data/{file_name}", header=True, inferSchema=True)

In [15]:
df = df.select('Price', 'Volume')

In [16]:
from pyspark.sql import functions as F
df = df.withColumn('idTransaction', F.monotonically_increasing_id())

In [18]:
uri = "jdbc:clickhouse://{}:{}/{}".format("localhost", 8123, "SgxTrading")

fact = {
            'url': uri,
            'dbtable' : 'DerivatiesTrading',
            'isolationLevel' : 'NONE'
        }
df.write.mode("append").format("jdbc").options(**fact).save()

Py4JJavaError: An error occurred while calling o56.save.
: java.sql.SQLException: No suitable driver
	at java.sql.DriverManager.getDriver(DriverManager.java:315)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:109)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:109)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:47)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:47)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.Ex